In [32]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score

st.set_page_config(page_title="Concrete Strength Prediction", page_icon="🏗️", layout="wide")

st.title("🏗️ Concrete Strength Prediction using CNN")
st.write("This AI/ML model predicts the **compressive strength** of concrete based on its composition using a **1D Convolutional Neural Network**.")

uploaded_file = st.file_uploader("📂 Upload your Concrete dataset (CSV)", type=["csv"])

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.success("✅ Dataset uploaded successfully!")
    st.dataframe(df.head())

    st.subheader("📊 Data Insights")

    # Select Target Column
    target_col = st.selectbox("🎯 Select Target Column (Strength Column):", df.columns)

    # Center alignment helper
    def center_plot(fig):
        col1, col2, col3 = st.columns([1, 3, 1])
        with col2:
            st.pyplot(fig)

    # Target Distribution
    st.write("### 🎯 Target Column Distribution")
    fig, ax = plt.subplots(figsize=(6,4))
    sns.histplot(df[target_col], kde=True, ax=ax)
    ax.set_xlabel(target_col)
    ax.set_ylabel("Frequency")
    center_plot(fig)

    # Correlation Heatmap
    st.write("### 🔥 Feature Correlation Heatmap")
    fig, ax = plt.subplots(figsize=(6,4))
    sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap="coolwarm", ax=ax)
    center_plot(fig)

    # Separate features & label
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Split + Scale
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # CNN expects 3D input
    X_train_scaled = np.expand_dims(X_train_scaled, axis=2)
    X_test_scaled = np.expand_dims(X_test_scaled, axis=2)

    # Build CNN Model
    model = Sequential([
        Conv1D(64, 2, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
        Conv1D(32, 2, activation='relu'),
        Dropout(0.2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(0.001), loss='mse')

    st.info("⏳ Training CNN model... Please wait (50 epochs)...")
    history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                        epochs=50, batch_size=16, verbose=0)
    st.success("✅ Model trained successfully!")

    preds = model.predict(X_test_scaled).flatten()

    # Metrics
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)

    col1, col2 = st.columns(2)
    with col1:
        st.metric("📉 RMSE", f"{rmse:.3f}")
    with col2:
        st.metric("📈 R² Score", f"{r2:.3f}")

    # Scatter Plot
    st.write("### 🎯 Actual vs Predicted Strength")
    fig, ax = plt.subplots(figsize=(6,4))
    ax.scatter(y_test, preds)
    ax.set_xlabel("Actual Strength")
    ax.set_ylabel("Predicted Strength")
    ax.set_title("Actual vs Predicted")
    center_plot(fig)

    # Residual Plot
    st.write("### ⚠️ Residual Error Plot")
    residuals = y_test - preds
    fig, ax = plt.subplots(figsize=(6,4))
    ax.scatter(preds, residuals)
    ax.axhline(0, color='red')
    ax.set_xlabel("Predicted Strength")
    ax.set_ylabel("Residual Error")
    ax.set_title("Residual Plot")
    center_plot(fig)

    # Training Progress
    st.subheader("📊 Training Progress")
    loss_df = pd.DataFrame({
       "Training Loss": history.history["loss"],
       "Validation Loss": history.history["val_loss"]
    })
    st.line_chart(loss_df)

    # Custom Prediction
    st.subheader("🔍 Predict Custom Input")
    input_values = []
    for col in X.columns:
        val = st.number_input(f"{col}", float(df[col].min()), float(df[col].max()), float(df[col].mean()))
        input_values.append(val)

    if st.button("🚀 Predict Strength"):
        sample = scaler.transform(np.array(input_values).reshape(1, -1))
        sample = np.expand_dims(sample, axis=2)
        pred = model.predict(sample).flatten()[0]
        st.success(f"🏗️ Predicted Concrete Strength: **{pred:.2f} MPa**")

else:
    st.warning("👆 Please upload a CSV dataset to begin.")


Overwriting app.py


In [28]:
!pip install tensorflow pandas numpy streamlit pyngrok scikit-learn


In [29]:
!ngrok config add-authtoken 34gxLRUIumdO3mTmb6lxfYXAhgv_53U9CB7pYz38w5jiPZawg

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [33]:
from pyngrok import ngrok
import time, subprocess

ngrok.kill()
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])
time.sleep(5)
public_url = ngrok.connect(8501)
print("🌐 Streamlit App URL:", public_url)


🌐 Streamlit App URL: NgrokTunnel: "https://venomously-unreflecting-queenie.ngrok-free.dev" -> "http://localhost:8501"
